# Préparation des données pour le classificateur de prunes africaines

Ce notebook utilise les fonctions existantes dans le dépôt pour préparer les données d'entraînement et de test du modèle de classification des prunes africaines.

## 1. Configuration de l'environnement pour Google Colab

Commençons par cloner le dépôt GitHub et configurer l'environnement.

In [ ]:
# Vérifier si nous sommes dans Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Exécution dans Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Cloner le dépôt GitHub
    !git clone https://github.com/CodeStorm-mbe/african-plums-classifier.git
    %cd african-plums-classifier
    
    # Installer les dépendances requises
    !pip install -r requirements.txt

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random

# Ajouter le répertoire courant au chemin pour pouvoir importer nos modules
if IN_COLAB:
    # Dans Colab, nous sommes déjà dans le répertoire du projet
    if "/content/african-plums-classifier" not in sys.path:
        sys.path.append("/content/african-plums-classifier")
else:
    # En local, ajouter le répertoire parent
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import (
    load_and_prepare_data,
    load_and_prepare_two_stage_data,
    visualize_batch,
    analyze_dataset_distribution
)

# Définir les chemins des données
if IN_COLAB:
    # Dans Colab, créer les répertoires dans le dossier du projet cloné
    DATA_ROOT = "data/raw"
else:
    # En local
    DATA_ROOT = "../data/raw"

PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes

# Vérifier si les répertoires existent
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(NON_PLUM_DATA_DIR, "non_plum"), exist_ok=True)

# Définir les paramètres
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_WORKERS = 2 if IN_COLAB else 4  # Réduire le nombre de workers dans Colab
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

## 2. Création de données d'exemple

Puisque nous travaillons dans Google Colab sans accès à vos fichiers locaux, créons des données d'exemple pour tester le notebook.

In [ ]:
def create_sample_data(force_create=False):
    """Crée des données d'exemple pour tester le notebook."""
    # Vérifier si des données existent déjà
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    
    if plum_classes and os.path.exists(non_plum_dir) and not force_create:
        print("Des données existent déjà. Utilisez force_create=True pour les remplacer.")
        return
    
    # Créer la structure de dossiers
    plum_classes = ["ripe", "unripe", "damaged", "diseased", "overripe", "healthy"]
    for cls in plum_classes:
        os.makedirs(os.path.join(PLUM_DATA_DIR, cls), exist_ok=True)
    
    os.makedirs(non_plum_dir, exist_ok=True)
    
    # Créer des images d'exemple (carrés colorés)
    colors = {
        "ripe": (150, 0, 0),      # Rouge foncé
        "unripe": (0, 150, 0),    # Vert foncé
        "damaged": (150, 100, 0), # Marron
        "diseased": (100, 0, 100),# Violet
        "overripe": (100, 0, 0),  # Rouge très foncé
        "healthy": (150, 50, 50)  # Rouge-rose
    }
    
    # Générer des images pour chaque classe de prune
    for cls, base_color in colors.items():
        for i in range(10):  # 10 images par classe
            # Ajouter un peu de variation aléatoire à la couleur
            color_var = [max(0, min(255, c + random.randint(-20, 20))) for c in base_color]
            
            # Créer une image
            img = Image.new('RGB', (224, 224), (255, 255, 255))
            pixels = img.load()
            
            # Dessiner un cercle approximatif avec la couleur
            center_x, center_y = 112, 112
            radius = 100 + random.randint(-10, 10)
            
            for x in range(img.width):
                for y in range(img.height):
                    dist = ((x - center_x) ** 2 + (y - center_y) ** 2) ** 0.5
                    if dist <= radius:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color_var]
                        pixels[x, y] = tuple(pixel_color)
            
            # Sauvegarder l'image
            img_path = os.path.join(PLUM_DATA_DIR, cls, f"{cls}_{i+1}.jpg")
            img.save(img_path)
    
    # Générer des images pour la classe non-prune
    for i in range(20):  # 20 images non-prune
        # Couleur aléatoire qui n'est pas proche des couleurs de prune
        color = (random.randint(0, 100), random.randint(150, 255), random.randint(150, 255))
        
        # Créer une image
        img = Image.new('RGB', (224, 224), (255, 255, 255))
        pixels = img.load()
        
        # Dessiner une forme aléatoire (carré ou triangle)
        shape = random.choice(['square', 'triangle'])
        
        if shape == 'square':
            # Dessiner un carré
            size = random.randint(100, 150)
            top_left = (random.randint(0, 224-size), random.randint(0, 224-size))
            
            for x in range(top_left[0], top_left[0] + size):
                for y in range(top_left[1], top_left[1] + size):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                        pixels[x, y] = tuple(pixel_color)
        else:
            # Dessiner un triangle
            p1 = (random.randint(50, 174), random.randint(50, 174))
            p2 = (p1[0] + random.randint(30, 50), p1[1] + random.randint(30, 50))
            p3 = (p1[0] - random.randint(0, 30), p2[1])
            
            # Remplir le triangle (algorithme simple)
            min_x = min(p1[0], p2[0], p3[0])
            max_x = max(p1[0], p2[0], p3[0])
            min_y = min(p1[1], p2[1], p3[1])
            max_y = max(p1[1], p2[1], p3[1])
            
            for x in range(min_x, max_x + 1):
                for y in range(min_y, max_y + 1):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Vérification simple si le point est dans le triangle
                        if (x >= p1[0] and y >= p1[1] and x <= p2[0] and y <= p2[1]):
                            # Ajouter du bruit à chaque pixel
                            pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                            pixels[x, y] = tuple(pixel_color)
        
        # Sauvegarder l'image
        img_path = os.path.join(non_plum_dir, f"non_plum_{i+1}.jpg")
        img.save(img_path)
    
    print(f"Données d'exemple créées avec succès!")
    print(f"- {len(plum_classes)} classes de prunes avec 10 images chacune")
    print(f"- 20 images non-prune")

# Créer des données d'exemple
create_sample_data(force_create=True)

## 3. Exploration des données

In [ ]:
# Fonction simple pour explorer les répertoires de données
def explore_directory(directory):
    if not os.path.exists(directory):
        print(f"Le répertoire {directory} n'existe pas.")
        return
    
    print(f"Contenu du répertoire {directory}:")
    subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    print(f"Sous-dossiers: {subdirs}")
    
    for subdir in subdirs:
        subdir_path = os.path.join(directory, subdir)
        files = [f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]
        print(f"  - {subdir}: {len(files)} fichiers")

# Explorer les répertoires de données
print("=== Exploration des données de prunes ===")
explore_directory(PLUM_DATA_DIR)

print("\n=== Exploration des données non-prunes ===")
explore_directory(NON_PLUM_DATA_DIR)

## 4. Analyse de la distribution des classes

Utilisons la fonction `analyze_dataset_distribution` du module `data_preprocessing` pour analyser la distribution des classes.

In [ ]:
# Analyser la distribution des classes
try:
    print("Analyse de la distribution des classes de prunes...")
    class_counts, distribution_img = analyze_dataset_distribution(PLUM_DATA_DIR)
    
    # Afficher les résultats
    print("\nDistribution des classes de prunes:")
    for cls, count in class_counts.items():
        print(f"  - {cls}: {count} images")
    
    # Afficher le graphique de distribution
    if os.path.exists('class_distribution.png'):
        plt.figure(figsize=(10, 6))
        img = plt.imread('class_distribution.png')
        plt.imshow(img)
        plt.axis('off')
        plt.title('Distribution des classes')
        plt.show()
except Exception as e:
    print(f"Erreur lors de l'analyse de la distribution des classes: {e}")

## 5. Préparation des données pour le modèle à deux étapes

Utilisons la fonction `load_and_prepare_two_stage_data` du module `data_preprocessing` pour préparer les données.

In [ ]:
# Vérifier si les répertoires de données existent et contiennent des images
def check_data_availability():
    # Vérifier le répertoire des prunes
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    if not plum_classes:
        print(f"Aucune classe de prune trouvée dans {PLUM_DATA_DIR}. Veuillez ajouter des données.")
        return False
    
    # Vérifier le répertoire des non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if not os.path.exists(non_plum_dir):
        print(f"Le répertoire {non_plum_dir} n'existe pas. Veuillez créer ce répertoire et y ajouter des images.")
        return False
    
    return True

# Vérifier la disponibilité des données
data_available = check_data_availability()

if data_available:
    try:
        # Charger et préparer les données pour les deux étapes
        print("Chargement des données pour les deux étapes...")
        (detection_train_loader, detection_val_loader, detection_test_loader, detection_class_names), \
        (classification_train_loader, classification_val_loader, classification_test_loader, classification_class_names) = \
            load_and_prepare_two_stage_data(
                PLUM_DATA_DIR, 
                NON_PLUM_DATA_DIR,
                batch_size=BATCH_SIZE, 
                img_size=IMG_SIZE,
                num_workers=NUM_WORKERS
            )
        
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        
        # Afficher les tailles des datasets
        print(f"\nTailles des datasets de détection:")
        print(f"  - Entraînement: {len(detection_train_loader.dataset)} images")
        print(f"  - Validation: {len(detection_val_loader.dataset)} images")
        print(f"  - Test: {len(detection_test_loader.dataset)} images")
        
        print(f"\nTailles des datasets de classification:")
        print(f"  - Entraînement: {len(classification_train_loader.dataset)} images")
        print(f"  - Validation: {len(classification_val_loader.dataset)} images")
        print(f"  - Test: {len(classification_test_loader.dataset)} images")
    except Exception as e:
        print(f"Erreur lors du chargement des données: {e}")
else:
    print("Veuillez d'abord ajouter des données dans les répertoires appropriés.")

## 6. Visualisation des données

Utilisons la fonction `visualize_batch` du module `data_preprocessing` pour visualiser un batch d'images.

In [ ]:
# Visualiser un batch d'images pour chaque étape
if data_available and 'detection_train_loader' in locals() and 'classification_train_loader' in locals():
    try:
        print("Visualisation d'un batch d'images pour la détection...")
        detection_batch_viz = visualize_batch(detection_train_loader, detection_class_names)
        
        print("\nVisualisation d'un batch d'images pour la classification...")
        classification_batch_viz = visualize_batch(classification_train_loader, classification_class_names)
        
        # Afficher les images sauvegardées
        if os.path.exists('batch_visualization.png'):
            plt.figure(figsize=(12, 10))
            img = plt.imread('batch_visualization.png')
            plt.imshow(img)
            plt.axis('off')
            plt.title('Visualisation du batch')
            plt.show()
    except Exception as e:
        print(f"Erreur lors de la visualisation des batches: {e}")

## 7. Conclusion

Dans ce notebook, nous avons utilisé les fonctions existantes du module `data_preprocessing` pour :
1. Explorer les données
2. Analyser la distribution des classes
3. Préparer les données pour le modèle à deux étapes
4. Visualiser les données

Les données sont maintenant prêtes pour l'entraînement du modèle dans le notebook suivant.